In [1]:
from io import StringIO
from bs4 import BeautifulSoup
from tika import parser
import re

file_path = "../Books Index for Phrases/2013_Bodie.Kane.Marcus_Investments (McGraw-Hill, 10th Ed).pdf"

file_data = []
_buffer = StringIO()
data = parser.from_file(file_path, xmlContent=True)
xhtml_data = BeautifulSoup(data['content'])
for page, content in enumerate(xhtml_data.find_all('div', attrs={'class': 'page'})):
    print('Parsing page {} of pdf file...'.format(page+1))
    _buffer = StringIO()
    _buffer.write(str(content))
    parsed_content = parser.from_buffer(_buffer.getvalue())
    _buffer.truncate()
    file_data.append({'id': 'page_'+str(page+1), 'content': parsed_content['content']})

Parsing page 1 of pdf file...
Parsing page 2 of pdf file...
Parsing page 3 of pdf file...
Parsing page 4 of pdf file...
Parsing page 5 of pdf file...
Parsing page 6 of pdf file...
Parsing page 7 of pdf file...
Parsing page 8 of pdf file...
Parsing page 9 of pdf file...
Parsing page 10 of pdf file...
Parsing page 11 of pdf file...
Parsing page 12 of pdf file...
Parsing page 13 of pdf file...
Parsing page 14 of pdf file...
Parsing page 15 of pdf file...
Parsing page 16 of pdf file...
Parsing page 17 of pdf file...
Parsing page 18 of pdf file...
Parsing page 19 of pdf file...
Parsing page 20 of pdf file...
Parsing page 21 of pdf file...
Parsing page 22 of pdf file...
Parsing page 23 of pdf file...
Parsing page 24 of pdf file...
Parsing page 25 of pdf file...
Parsing page 26 of pdf file...
Parsing page 27 of pdf file...
Parsing page 28 of pdf file...
Parsing page 29 of pdf file...
Parsing page 30 of pdf file...
Parsing page 31 of pdf file...
Parsing page 32 of pdf file...
Parsing page 33 o

## Glossary

In [2]:
## Glossary ## 
## Page 2 (G-1) to Page 15 (G-14)
glossary_pages = xhtml_data.find_all('div', attrs={'class': 'page'})[1: 15]


In [3]:
glossary_page_1 = glossary_pages[0]

In [4]:
for p in glossary_page_1.findAll(name='p'):
    # print(p.text)
    text = p.text
    text = text.replace('’', '')
    ## remove page number
    if re.search(r'G-\d{1,2}', text): 
        continue

    ## remove Glossary 
    if re.search(r'Glossary', text): 
        continue

    ## remove Final PDF to printer
    if re.search(r'Final PDF to printer', text): 
        continue
    
    text = text.replace('\n', '').strip()
    text = text.split('    ')[-2:]
    
    if text != ['']:
        print(text)

['arbitrage pricing theory', 'An asset pricing theory that is derived from a factor model, using diversification and arbitrage arguments. The theory describes the relationship between expected returns on securities, given that there are no opportunities to create wealth through risk-free arbitrage investments.']
['ask price', 'The price at which a dealer will sell a security.']
['asset allocation', 'Choosing among broad asset classes such as stocks versus bonds.']
['at the money', 'When the exercise price and asset price of an option are equal.']
['auction market', 'A market where all traders in a good meet at one place to buy or sell an asset. The NYSE is an example.']
['average collection period, or days receivables', 'The ratio of accounts receivable to sales, or the total amount of credit extended per dollar of daily sales (average AR/sales\xa0 3 \xa0365).']
['backfill bias', 'Bias in the average returns of a sample of funds induced by including past returns on funds that entered t

In [5]:
TEXT = []
PHRASES = []
EXPLANATION = []

for page in glossary_pages:

    for p in page.findAll(name='p'):
        # print(p.text)
        text = p.text
        text = text.replace('’', "'")
        text = text.replace('–', '-')
        ## remove page number
        if re.search(r'G-\d{1,2}', text): 
            continue

        ## remove Glossary 
        if re.search(r'Glossary', text): 
            continue

        ## remove Final PDF to printer
        if re.search(r'Final PDF to printer', text): 
            continue
        
        
        text = text.replace('\n', '').strip()
        text_original = text
        text = text.split('    ')[-2:]
        
        if text != ['']:
            # print(text)
            phrase, explanation = text
            phrase = phrase.replace('"', '').replace('  ', ' ').strip()
            explanation = explanation.replace('"', '').replace('  ', ' ').strip()
            TEXT.append(text_original)
            PHRASES.append(phrase)
            EXPLANATION.append(explanation)

In [6]:
import pandas as pd 
df = pd.DataFrame({'ID': [i for i in range(len(PHRASES))],
            'original_glossary_extracted': TEXT,
            'key_phrase': PHRASES,
            'source_book': ['2013_Bodie.Kane.Marcus_Investments' for _ in range(len(PHRASES))],
            'definition_or_explanation': EXPLANATION})

In [7]:
# df.to_csv('glossary_extraction_2013_Bodie.Kane.Marcus_Investments.csv', index=False)

In [8]:
df.head()

,ID,original_glossary_extracted,key_phrase,source_book,definition_or_explanation
0,0,arbitrage pricing theory An asset pricing t...,arbitrage pricing theory,2013_Bodie.Kane.Marcus_Investments,An asset pricing theory that is derived from a...
1,1,ask price The price at which a dealer will ...,ask price,2013_Bodie.Kane.Marcus_Investments,The price at which a dealer will sell a security.
2,2,asset allocation Choosing among broad asset...,asset allocation,2013_Bodie.Kane.Marcus_Investments,Choosing among broad asset classes such as sto...
3,3,at the money When the exercise price and as...,at the money,2013_Bodie.Kane.Marcus_Investments,When the exercise price and asset price of an ...
4,4,auction market A market where all traders i...,auction market,2013_Bodie.Kane.Marcus_Investments,A market where all traders in a good meet at o...


## Reformat extraction

In [9]:
import re
def find_full_expression_and_abbreviation(text):
    text_length = len(text)
    match = re.search('\(.+\)', text) # search of bracket
    match_1 = re.search('[A-Z]{2,8}', text) # search for Capitalization
    match_2 = re.search('[A-Z]/[A-Z]', text) # search for Capitalization
    if match and (match_1 or match_2):
        content_in_bracket_length = len(match.group(0)) - 2
    else:
        if re.search('\(.+', text):
            pos = re.search('\(.+', text).span()[0]
            return [text[:pos-1]]

        else:
            return [text]

    start, end = re.search('\(.+\)', text).span()
    if content_in_bracket_length > text_length / 2:
        
        full_expression = text[start+1: end-1]
        abbreviated_expression = text[: start-1]
    else:
        full_expression = text[: start-1]
        abbreviated_expression = text[start+1: end-1]

    return [full_expression, abbreviated_expression]

In [10]:
full = []
extracted_acronym_1 = []
extracted_acronym_2 = []

for phrase, definition  in zip(df['key_phrase'].values, df['definition_or_explanation'].values):
     res = find_full_expression_and_abbreviation(phrase)
     if len(res) == 1:
         full.append(res[0])
         extracted_acronym_1.append('')
         extracted_acronym_2.append('')

     if len(res) == 2:
         full.append(res[0])
         extracted_acronym_1.append(res[1])
         extracted_acronym_2.append('')

In [11]:
df['key_phrase'] = full
df['extracted_acronym_1'] = extracted_acronym_1
df['extracted_acronym_2'] = extracted_acronym_2

In [16]:
def clean_extracted_key_phrase(phrase):
    phrase = phrase.replace('-', ' ')
    phrase = phrase.replace('/', ' ')
    return phrase

In [17]:
df['key_phrase'] = df['key_phrase'].apply(clean_extracted_key_phrase)

In [18]:
df.columns

Index(['ID', 'original_glossary_extracted', 'key_phrase',
       'extracted_acronym_1', 'extracted_acronym_2',
       'definition_or_explanation', 'source_book'],
      dtype='object')

In [19]:
df = df[['ID', 'original_glossary_extracted', 'key_phrase', 'extracted_acronym_1',
       'extracted_acronym_2', 'definition_or_explanation', 'source_book',
        ]]

In [20]:
df.to_csv('glossary_extraction_2013_Bodie.Kane.Marcus_Investments.csv', index=False)